In [1]:
import pandas as pd
import numpy as np

In [22]:
## r2py setup
import rpy2.rinterface_lib.callbacks
import logging
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

In [23]:
%load_ext rpy2.ipython

In [24]:
%%R
library(tidyverse)

System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to create bus connection: Host is down


In [31]:
HCA_manifest = pd.read_table('/nfs/team205/ed6/data/hca-manifest-6c5304b0-336f-5efe-8302-a724e06e59d2.83e2ac44-854a-54ee-9420-b4e2f6102404.tsv', index_col=0)

In [32]:
HCA_manifest.columns

Index(['source_spec', 'bundle_uuid', 'bundle_version', 'file_document_id',
       'file_type', 'file_name', 'file_format', 'read_index', 'file_size',
       'file_uuid', 'file_version', 'file_crc32c', 'file_sha256',
       'file_content_type', 'file_drs_uri', 'file_url',
       'cell_suspension.provenance.document_id',
       'cell_suspension.biomaterial_core.biomaterial_id',
       'cell_suspension.estimated_cell_count',
       'cell_suspension.selected_cell_type',
       'sequencing_process.provenance.document_id',
       'sequencing_protocol.instrument_manufacturer_model',
       'sequencing_protocol.paired_end',
       'library_preparation_protocol.library_construction_approach',
       'library_preparation_protocol.nucleic_acid_source',
       'project.provenance.document_id', 'project.contributors.institution',
       'project.contributors.laboratory',
       'project.project_core.project_short_name',
       'project.project_core.project_title', 'project.estimated_cell_count',
  

In [84]:
HCA_manifest.file_content_type.unique()

array(['application/octet-stream; dcp-type=data; dcp-type=data',
       'text/tab-separated-values; dcp-type=data; dcp-type=data',
       'application/vnd.loom; dcp-type=data',
       'application/gzip; dcp-type=data; dcp-type=data',
       'binary/octet-stream; dcp-type=data; dcp-type=data',
       'application/x-tar; dcp-type=data; dcp-type=data',
       'application/zip; dcp-type=data; dcp-type=data',
       'text/csv; dcp-type=data; dcp-type=data',
       'text/plain; dcp-type=data; dcp-type=data',
       'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet; dcp-type=data; dcp-type=data'],
      dtype=object)

In [59]:
project_cols = ['project.project_core.project_short_name']
donor_cols = HCA_manifest.columns[HCA_manifest.columns.str.startswith('donor')].tolist()
specimen_cols = [
    'specimen_from_organism.provenance.document_id',
     'specimen_from_organism.diseases',
 'specimen_from_organism.organ',
 'specimen_from_organism.organ_part'
]

In [69]:
HCA_manifest_keep = HCA_manifest[project_cols + donor_cols ]
HCA_manifest_keep.loc[HCA_manifest_keep['donor_organism.organism_age'].isna(), 'donor_organism.organism_age'] = 'embryonic'
HCA_manifest_keep.loc[HCA_manifest_keep['donor_organism.diseases'].isna(), 'donor_organism.diseases'] = 'normal'

/tmp/ipykernel_8700/77444952.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HCA_manifest_keep.loc[HCA_manifest_keep['donor_organism.organism_age'].isna(), 'donor_organism.organism_age'] = 'embryonic'
/tmp/ipykernel_8700/77444952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HCA_manifest_keep.loc[HCA_manifest_keep['donor_organism.diseases'].isna(), 'donor_organism.diseases'] = 'normal'


In [76]:
HCA_manifest_keep['donor_organism.sex'].unique()

array(['unknown', 'female', 'female || male', 'male',
       'female || male || unknown', 'male || unknown'], dtype=object)

In [70]:
%%R -i HCA_manifest_keep
head(HCA_manifest_keep)

  project.project_core.project_short_name donor_organism.sex
1                    myocardialInfarction            unknown
2           pulmonaryFibrosisPathobiology             female
3         HumanNaturalKillerDiversityYang             female
4                         1M Immune Cells     female || male
5             spatiotemporalIntestinalDev     female || male
6                         1M Immune Cells             female
        donor_organism.biomaterial_core.biomaterial_id
1                                                  P22
2                                              donor_1
3                                              Donor_2
4 CB1 || CB2 || CB3 || CB4 || CB5 || CB6 || CB7 || CB8
5                      AAB || AAE || AAF || AAT || ABV
6                                                  CB3
                                                                                                                                                                                            

/home/jovyan/my-conda-envs/oor-benchmark/lib/python3.10/site-packages/rpy2/robjects/pandas2ri.py:56: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


In [74]:
%%R
## Find collapsed columns
cols <- colnames(HCA_manifest_keep)

HCA_manifest_keep %>%
    head() 
#     separate_rows(cols, convert=TRUE) 
#     mutate(specimen_from_organism.diseases = strsplit(as.character(specimen_from_organism.diseases), ' || ')) %>%
#     head() %>%
#     unnest(specimen_from_organism.diseases)

  project.project_core.project_short_name donor_organism.sex
1                    myocardialInfarction            unknown
2           pulmonaryFibrosisPathobiology             female
3         HumanNaturalKillerDiversityYang             female
4                         1M Immune Cells     female || male
5             spatiotemporalIntestinalDev     female || male
6                         1M Immune Cells             female
        donor_organism.biomaterial_core.biomaterial_id
1                                                  P22
2                                              donor_1
3                                              Donor_2
4 CB1 || CB2 || CB3 || CB4 || CB5 || CB6 || CB7 || CB8
5                      AAB || AAE || AAF || AAT || ABV
6                                                  CB3
                                                                                                                                                                                            